In [1]:
import torch
import torch.sparse as sparse
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy.sparse.linalg
import math
import numpy as np
import scipy 
import importlib
import time
import torch.sparse as sparse
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy.sparse.linalg
import math
import numpy as np
import scipy 
import scipy.io
import matplotlib.pyplot as plt
from functools import partial
import time
import sys
import os
import gc
import importlib
import RFrtModel_SingleFracture_SolverDA_General as RF_1Frac_DA

In [4]:
importlib.reload(RF_1Frac_DA)

<module 'RFrtModel_SingleFracture_SolverDF' from '/Users/huynh/Desktop/Python/UnitedFilter_SingleRFrt/General/RFrtModel_SingleFracture_SolverDF.py'>

## Reverse SDE:

In [6]:
def cond_alpha(t):
    # conditional information
    # alpha_t(0) = 1
    # alpha_t(1) = esp_alpha \approx 0
    return 1 - (1-eps_alpha)*t


def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def f(t):
    # f=d_(log_alpha)/dt
    alpha_t = cond_alpha(t)
    f_t = -(1-eps_alpha) / alpha_t
    return f_t

def g_sq(t):
    # g = d(sigma_t^2)/dt -2f sigma_t^2
    d_sigma_sq_dt = 1
    g2 = d_sigma_sq_dt - 2*f(t)*cond_sigma_sq(t)
    return g2

def g(t):
    return np.sqrt(g_sq(t))

def reverse_SDE(DOFs_Local, NumEgs, NumEms, NeumannEgs, n_dim, select_set, idA_sub, x0, time_steps, C,\
                score_likelihood=None, drift_fun=f, diffuse_fun=g, alpha_fun=cond_alpha, \
                sigma2_fun=cond_sigma_sq, save_path=False):
    # x_T: sample from standard Gaussian
    # x_0: target distribution to sample from

    # reverse SDE sampling process
    # N1 = x_T.shape[0]
    # N2 = x0.shape[0]
    # d = x_T.shape[1]

    # Generate the time mesh
    dt = 1.0/time_steps

    # Initialization
    xt = torch.randn(ensemble_size, n_dim, dtype=torch.float64, device=device)
    t = 1.0

    # define storage
    if save_path:
        path_all = [xt]
        t_vec = [t]

    # forward Euler sampling
    for i in range(time_steps):
        # prior score evaluation
        alpha_t = alpha_fun(t)
        sigma2_t = sigma2_fun(t)


        # Evaluate the diffusion term
        diffuse = diffuse_fun(t)

        # Evaluate the drift term
        # drift = drift_fun(t)*xt - diffuse**2 * score_eval

        # Update
        if score_likelihood is not None:
#             zt = score_likelihood(xt, t)
#             print(zt.size())
            xt += -dt*( drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t)-\
                       diffuse**2*score_likelihood(idA_sub, xt, t, C)) +np.sqrt(dt)*diffuse*torch.randn_like(xt)
    
        else:
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t))+np.sqrt(dt)*diffuse*torch.randn_like(xt)
        
        xt = xt.to(torch.float64)
        # Store the state in the path
        if save_path:
            path_all.append(xt)
            t_vec.append(t)

        # update time
        t = t - dt

    if save_path:
        return path_all, t_vec
    else:
        return xt

In [2]:
def systematic_resample_pytorch(weights):
    N = len(weights)
    
    # Generate positions with a consistent random offset
    positions = (torch.rand(1) + torch.arange(N)) / N

    # Compute cumulative sum of weights
    cumulative_sum = torch.cumsum(weights, dim=0)

    # Initialize the indexes tensor
    indexes = torch.zeros(N, dtype=torch.long)
    
    # Initialize pointers
    i, j = 0, 0
    while i < N and j < len(cumulative_sum):
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    
    return indexes

In [3]:
xt = 0
xe = 2
yt = 0
ye = 1
nx = 80
ny = 40
node_frt = ny + 1
device = 'cpu'
# device = 'cuda'
nt = 800
T = 1
t0 = 0
SDE_sigma = 0.005

h = (ye - yt) / ny
dt = (T - t0) / nt

frt_length = 1e-03

timegrid = torch.linspace(0, T, nt + 1)
poros_subdom = 1
poros_gamma = 1 / frt_length
poros_frt = frt_length * poros_gamma

# Mesh generation and diffusion parameters
TriMesh_Subdom = RF_1Frac_DA.MeshGenerator_PureDiff(xt, xe, yt, ye, nx, ny, device)
n_dim = TriMesh_Subdom[0].DOFs+TriMesh_Subdom[1].DOFs+2*ny+1
DOFs_Local = [None, None]
NumEgs = [None, None]
NumEms = [None, None]
for k in range(2):
    DOFs_Local[k] = TriMesh_Subdom[k].DOFs
    NumEgs[k] = TriMesh_Subdom[k].NumEgs
    NumEms[k] = TriMesh_Subdom[k].NumEms
# print(TriMesh_Subdom[0].DirichletEgs.shape)

DOFs = DOFs_Local[0]+DOFs_Local[1]+node_frt+ny
diff_1subdom = 1
diff_2subdom = 1

diff_frt_normal = torch.zeros(ny, dtype=torch.float64)
diff_frt_tangent = torch.zeros(ny, dtype=torch.float64)

kf = 2000
# Assign values to segments of diff_frt
diff_frt_normal[:ny // 4] = 1 / kf
diff_frt_normal[ny // 4:3 * ny // 4] = kf
diff_frt_normal[3 * ny // 4:] = 1 / kf

diff_frt_tangent[:ny // 4] = kf 
diff_frt_tangent[ny // 4:3 * ny // 4] = 1 / kf
diff_frt_tangent[3 * ny // 4:] = kf

# Calculate keppa and alp_frt
keppa = (2 * diff_frt_normal) / frt_length
alp_frt = diff_frt_tangent * frt_length

diffusion_ref = RF_1Frac_DA.compute_diffusion_single(diff_1subdom, diff_2subdom, TriMesh_Subdom, device)

GAUSSQUAD = RF_1Frac_DA.SetGaussQuad(2, 4, 3, device)
Eqn_Subdom = [RF_1Frac_DA.EqnBC(), RF_1Frac_DA.EqnBC()]
EqnBC_fracture = RF_1Frac_DA.EqnBC_frt()
Local_Mat = [None]*2
Local_Mat_pertb = [None]*2

for k in range(2):
    Local_Mat[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat(TriMesh_Subdom[k], dt, diffusion_ref[k], poros_subdom, device)
    
MatB = [None]*2
MatC = [None]*2

MatATest = [None]*2
Local_MatTest = [None]*2

for k in range(2):
    MatB[k], MatC[k] = RF_1Frac_DA.ComputeBandCblocks(TriMesh_Subdom[k], dt, poros_subdom,  device)
    
for k in range(2):
    MatATest[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dt, diffusion_ref[k], device)
    Local_MatTest[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatATest[k], MatB[k], MatC[k], dt)    

Glob_MatTest = RF_1Frac_DA.StiffMax_PureDiff_Vectorized_General(TriMesh_Subdom, h, node_frt, ny, \
                                                                Local_MatTest, alp_frt, keppa, dt, \
                                                                poros_frt, device)

BCs_frt = 3
BCs_Subdom = 3
X0 = torch.zeros((1, TriMesh_Subdom[0].DOFs+TriMesh_Subdom[1].DOFs+2*ny+1), dtype=torch.float64, device=device)

X0 = X0.to(device)
state_target = X0
# state_ref = torch.zeros(nt+1, n_dim, device = device)
# state_ref[[0], :] += state_target

# for i in range(nt):
#     print(i)
#     noise2 = np.sqrt(dt)*SDE_sigma*torch.randn_like(torch.zeros((NumEms[0]+NumEms[1]+ny, 1),\
#                                                                 dtype=torch.float64, device=device))
#     stateT = torch.transpose(state_ref[[i], :], 0, 1)
#     sln = RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA_General_Aniso(stateT, ny, node_frt, TriMesh_Subdom, nt, \
#                                                                          dt, poros_subdom, poros_frt, BCs_frt, BCs_Subdom,\
#                                                                          Glob_MatTest, GAUSSQUAD, Eqn_Subdom, \
#                                                                          EqnBC_fracture, SDE_sigma, device, noise2, 0)
#     state_ref[[i+1], :] += torch.transpose(sln, 0, 1)

# Save reference solution in .mat format

In [64]:
state_ref_save = state_ref.clone()
state_ref_save = state_ref.cpu().numpy()

scipy.io.savemat('TestEnSF_RefSol_General_Aniso_v1.mat', {'ExactState':state_ref_save, 'nx':nx, 'ny':ny, 'nt':nt})

## Choosing randomly the spatial locations for arctangent

In [5]:
TotalEms = TriMesh_Subdom[0].NumEms+TriMesh_Subdom[1].NumEms+ny
TotalEgs = TriMesh_Subdom[0].NumEgs+TriMesh_Subdom[1].NumEgs+node_frt

TotalEgsSubdom = TriMesh_Subdom[0].NumEgs+TriMesh_Subdom[1].NumEgs
TotalEmsSubdom = TriMesh_Subdom[0].NumEms+TriMesh_Subdom[1].NumEms
#### 50%
indices_A = torch.randperm(TotalEgsSubdom)[:int(0.5 * TotalEgsSubdom)]
indices_Afrt = torch.randperm(node_frt)[:int(0.5 * node_frt)]+TotalEgsSubdom

indices_B = torch.randperm(TotalEmsSubdom)[:int(0.5 * TotalEmsSubdom)] + TotalEgs
indices_Bfrt = torch.randperm(ny)[:int(0.5 * ny)] + TotalEgs+TotalEmsSubdom

spa_indices, _ = torch.sort(torch.cat([indices_A, indices_Afrt, indices_B, indices_Bfrt]))

num_indices= indices_A.shape[0]+indices_Afrt.shape[0]+indices_B.shape[0]+indices_Bfrt.shape[0]

r1 = indices_B.shape[0] / indices_A.shape[0]
r2 = indices_Afrt.shape[0]/ indices_A.shape[0]
r3 = indices_Bfrt.shape[0]/ indices_A.shape[0]

x = 50*(r1+r2+r3+1)

idA_sub = torch.randperm(len(indices_A))[:int(round((x / 100) * len(indices_A)))]

# # Sort the selected indices in increasing order
idA_sub, _ = torch.sort(idA_sub)

## Setting for the United Filter

In [270]:
# data = scipy.io.loadmat('TestEnSF_RefSol_General_Aniso_v2.mat')
# state_ref = torch.from_numpy(data['ExactState'])

n_dim = TriMesh_Subdom[0].DOFs+TriMesh_Subdom[1].DOFs+2*ny+1

ntEnSF = 50
SDE_Sigma = 0.001
#SDE_Sigma = 0.01
#SDE_Sigma = 0.1

filtering_steps = ntEnSF
timeTrue = torch.linspace(0, 1, nt+1)
tEnSF = torch.linspace(0, 1, filtering_steps+1)
indices = torch.searchsorted(timeTrue, tEnSF, right=False)

state_EnSF = state_ref[indices, :].clone()

state_EnSF_Swap = RF_1Frac_DA.RearrangeSol(state_EnSF, TriMesh_Subdom, ny, node_frt)
dtEnSF = (T - t0) / ntEnSF
obs_sigma = 0.2
####################################################################
# EnSF setup
# define the diffusion process
eps_alpha = 0.05

# ensemble size
ensemble_size = 200
ensemble_true = 1
# forward Euler step
euler_steps = 200
sub_steps = 4

time_euler = torch.linspace(0, 1, euler_steps+1)
time_sub = torch.linspace(0, 1, sub_steps+1)
select_set = torch.searchsorted(time_euler, time_sub, right=False)

print(select_set)
# device = 'cpu'
# device = 'cuda'

def g_tau(t):
    return 1-t

# saving file name
exp_name = 'UnitedF_RFrtModel_Mixed_General_Aniso'

x_state = 7*torch.randn(ensemble_size, n_dim, device=device)
# x_state = x_state.to(torch.float16)
mem_state = state_target.element_size() * state_target.nelement()/1e+6
mem_ensemble = mem_state * ensemble_size
print(f'single state memory: {mem_state:.2f} MB')
print(f'state ensemble memory: {mem_ensemble:.2f} MB')

rmse_all = []
obs_save = []
est_save = torch.zeros(filtering_steps+1, n_dim, device = device)
est_save[[0], :] += torch.mean(x_state,dim=0)

x0filter = x_state

# SlnRefT = torch.transpose(ExtractSlnRef, 0, 1)
X = torch.zeros((ntEnSF+1, DOFs), dtype=torch.float64, device=device) #545 is the dof in the mesh considered
X[0, :] += state_EnSF[0, :]
for i in range(1, ntEnSF+1):
    X[i, :] += state_EnSF[i, :]
    
state_United = state_EnSF_Swap[:, spa_indices].clone()

state_United = state_United.to(torch.float64)
factorR = 0.25
R = factorR*torch.eye(DOFs, dtype=torch.float64, device=device)   # Covariance considered 
cholR = torch.linalg.cholesky(R)
factor = 100
R_scaled = factor * R
Z = cholR[torch.meshgrid(spa_indices, spa_indices, indexing ='ij')] @ \
    torch.randn(num_indices, ntEnSF+1, dtype=torch.float64, device=device)

state_United += torch.transpose(Z, 0, 1)

NeumannEgs = [None]*2
for k in range(2):
    NeumannEgs[k] = TriMesh_Subdom[k].NeumannEgs

LocalMat_DF = [None]*2

MatD = RF_1Frac_DA.ComputeDblocks(TriMesh_Subdom, h, dtEnSF, ny, device)

theta0 = torch.tensor([8, 8, 4], dtype=torch.float64, device =device); #Initial guess
n = 50;                     # Number of particles considered     
Cov = torch.eye(3, dtype=torch.float64, device =device) * torch.tensor([4, 4, 1], dtype=torch.float64, device =device)
l= 4
particles =torch.distributions.MultivariateNormal(theta0, Cov).sample((n,))
xs = [theta0]
weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
IntmediaPara = torch.sum(particles * weights.unsqueeze(1), dim=0) 

tensor([  0,  50, 100, 150, 200])
single state memory: 0.13 MB
state ensemble memory: 25.99 MB


In [ ]:
for i in range(filtering_steps):
# for i in range(2):
    # weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
    print(f'step={i}:')
    t1 = time.time()
    obs = state_United[[i+1], :].clone()
    obs[:, idA_sub] = torch.atan(state_United[[i+1], idA_sub].clone())
    obs += torch.randn_like(state_United[[i+1], :])*obs_sigma
    def score_likelihood(idA_sub, xt, t, C):
        # obs: (d)
        # xt: (ensemble, d)
        A = -(xt-obs)/obs_sigma**2
        score_x = A.clone()
        score_x[:, idA_sub] = -(torch.atan(xt[:, idA_sub]) - obs[:, idA_sub])/obs_sigma**2 * (1./(1+xt[:, idA_sub]**2))
        tau = g_tau(t)
        return tau*score_x/C
    
    x0EnSF = x_state.clone()
    x0filterT = torch.transpose(x0EnSF.clone(), 0, 1)
    for j in range(l):
        weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
        IntmediaPara_Start = IntmediaPara.clone()
        ##### EnSF 
        Local_Matloop = [None]*2
        MatAEnSF = [None]*2
        
        diffusion_loopEnSF = RF_1Frac_DA.compute_diffusion_single(IntmediaPara_Start[0], IntmediaPara_Start[1],\
                                                                  TriMesh_Subdom, device)
        for k in range(2):
            MatAEnSF[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dtEnSF, diffusion_loopEnSF[k], device)
            Local_Matloop[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatAEnSF[k], MatB[k], MatC[k], dtEnSF)

        kfDA1 = IntmediaPara_Start[2]*2000
        
        diff_frt_normalDA = torch.zeros(ny, dtype=torch.float64)
        diff_frt_tangentDA = torch.zeros(ny, dtype=torch.float64)
        
        # Assign values to segments of diff_frt
        diff_frt_normalDA[:ny // 4] = 1 / kfDA1
        diff_frt_normalDA[ny // 4:3 * ny // 4] = kfDA1
        diff_frt_normalDA[3 * ny // 4:] = 1 / kfDA1
        
        diff_frt_tangentDA[:ny // 4] = kfDA1
        diff_frt_tangentDA[ny // 4:3 * ny // 4] = 1 / kfDA1
        diff_frt_tangentDA[3 * ny // 4:] = kfDA1
        
        # Calculate keppa and alp_frt
        keppaDA = (2 * diff_frt_normalDA) / frt_length
        alp_frtDA = diff_frt_tangentDA * frt_length
        
        Glob_Mat_loop = RF_1Frac_DA.StiffMax_PureDiff_Vectorized_General(TriMesh_Subdom, h, node_frt, ny, \
                                                                         Local_Matloop, alp_frtDA, keppaDA, dtEnSF, \
                                                                         poros_frt, device)
        
        sln_state = \
            RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA_General_Aniso(x0filterT, ny, node_frt, TriMesh_Subdom, ntEnSF,\
                                                                           dtEnSF, poros_subdom, poros_frt, BCs_frt, \
                                                                           BCs_Subdom, Glob_Mat_loop, GAUSSQUAD, Eqn_Subdom,\
                                                                           EqnBC_fracture, SDE_sigma, device, noise1, 0)
        
        x_state = torch.transpose(sln_state, 0, 1)+\
                    np.sqrt(dtEnSF)*SDE_Sigma*torch.randn_like(x_state, dtype=torch.float64, device=device)
        
        x_state = RF_1Frac_DA.RearrangeSol(x_state, TriMesh_Subdom, ny, node_frt)
        
        sln_bar = reverse_SDE(DOFs_Local, NumEgs, NumEms, NeumannEgs, num_indices, select_set, idA_sub,\
                              x0=x_state[:, spa_indices], time_steps=euler_steps, C=1, \
                              score_likelihood=score_likelihood)
        
        x_state[:, spa_indices] = sln_bar.clone()
        x_state[:, TotalEgs + torch.arange(0, NumEms[0])] =\
                torch.clamp(x_state[:, TotalEgs + torch.arange(0, NumEms[0])],\
                            min=0, max = 1.45)
    
        x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])] =\
                    torch.clamp(x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])],\
                                min=0, max = 2.99999)
    
        x_state[:, TotalEgs + NumEms[0] + NumEms[1]+ torch.arange(0, ny)] =\
                    torch.clamp(x_state[:, TotalEgs + NumEms[0] + NumEms[1]+ torch.arange(0, ny)],\
                                min=0, max = BCs_frt)
        
        x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-6, max = 6)
        x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-6, max=6)
        x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
                        min=-0.75, max = 9e-05)
            
        x_state[:, NeumannEgs[0]] = 0
        x_state[:, NumEgs[0]+NeumannEgs[1]] = 0
        
        x_state = RF_1Frac_DA.RearrangeSol_Reverse(x_state, TriMesh_Subdom, ny, node_frt)
        ####     
        xstate_bar = torch.mean(x_state,dim=0) # \bar{X}^{(l)}_{i+1}
        
        ## Adjusting the entries of Ep if needed, based on the value of SDE_Sigma.
        if i <= int(filtering_steps // 2):
            Ep = torch.diag(torch.tensor([0.81, 0.16, 0.64], dtype=torch.float64))  
        elif i < 4*int(filtering_steps // 5) and i > int(filtering_steps // 2):
            Ep = torch.diag(torch.tensor([0.01, 0.01, 0.005], dtype=torch.float64))            
        else:
            Ep = torch.diag(torch.tensor([0.005, 0.005, 0.001], dtype=torch.float64))

        cholEp = torch.linalg.cholesky(Ep)
        particles += torch.randn(n, 3, dtype=torch.float64)@cholEp #\tilde{\gamma}^{k, (l+1)}_{i+1}
        
        ## Avoid getting negative permeability
        for qq in range(particles.shape[1]):  # Iterate over columns
            condition = particles[:, qq] < 0  # Check for negative values in the column
            if condition.any():  # If there are negative values
                print(qq)
                print(f"negative detected in {'d2' if qq == 1 else 'd1' if qq == 0 else 'd3'}")
                # Find the smallest positive entry in the column
                positive_values = particles[:, qq][particles[:, qq] > 0]
                if positive_values.numel() > 0:  # Ensure there are positive values
                    smallest_positive = positive_values.min()
                    # Replace negative values with the smallest positive entry
                    particles[:, qq] = torch.where(
                        condition,
                        smallest_positive,
                        particles[:, qq]
                    )
#         z = xstate_bar.clone() #    

        z = xstate_bar.clone() # 
        P = torch.zeros(n, DOFs, dtype=torch.float64, device=device)

        x0DFT = torch.transpose(est_save[[i], :].clone(), 0, 1)
     
        for o, particle in enumerate(particles):            
            MatADF = [None]*2
            diff_DF = RF_1Frac_DA.compute_diffusion_single(particle[0], particle[1], TriMesh_Subdom, device)
  
            for k in range(2):
                MatADF[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dtEnSF, diff_DF[k], device)
                LocalMat_DF[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatADF[k], MatB[k], MatC[k], dtEnSF) 

            kfDF1 = particle[2]*2000
            # kfDF2 = particle[3]
            
            diff_frt_normalDF = torch.zeros(ny, dtype=torch.float64)
            diff_frt_tangentDF = torch.zeros(ny, dtype=torch.float64)
        
            # Assign values to segments of diff_frt
            diff_frt_normalDF[:ny // 4] = 1 / kfDF1
            diff_frt_normalDF[ny // 4:3 * ny // 4] = kfDF1
            diff_frt_normalDF[3 * ny // 4:] = 1 / kfDF1
            
            diff_frt_tangentDF[:ny // 4] = kfDF1
            diff_frt_tangentDF[ny // 4:3 * ny // 4] = 1 / kfDF1
            diff_frt_tangentDF[3 * ny // 4:] = kfDF1
            
            # Calculate keppa and alp_frt
            keppaDF = (2 * diff_frt_normalDF) / frt_length
            alp_frtDF = diff_frt_tangentDF * frt_length
        
            GlobMatDF = \
            RF_1Frac_DA.StiffMax_PureDiff_Vectorized_General(TriMesh_Subdom, h, node_frt, ny, \
                                                             LocalMat_DF, alp_frtDF, keppaDF, dtEnSF, \
                                                             poros_frt, device)
            
            SlnDF = \
            RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA_General_Aniso(x0DFT, ny, node_frt, TriMesh_Subdom, ntEnSF, \
                                                                           dtEnSF, poros_subdom, poros_frt, BCs_frt, BCs_Subdom, \
                                                                           GlobMatDF, GAUSSQUAD, Eqn_Subdom, EqnBC_fracture,\
                                                                           SDE_sigma, device, noise1, 0)
            SlnDFT = torch.transpose(SlnDF, 0, 1)
            P[o, :] += SlnDFT[-1, :] # \bar{X}^{\tilde{\gamma}^{k, (l+1)}_{i+1}}_{i+1}

        P -=  z.unsqueeze(0)   # Broadcasting is already efficient

        # Step 2: Solve R * X = P^T for X
        Rsolved = P*(1/factorR)
        Rnorm = torch.sum(Rsolved * P, dim=1)
        Rnorm_clone = Rnorm.clone() / 1
       
        if torch.min(Rnorm_clone) > 1100:
            alp_factor = torch.min(Rnorm_clone) / 1100 
#             print(alp_factor)
            Rnorm_clone /= alp_factor 
                    
        # print(Rnorm_clone)

        # Step 4: Update weights
        weights.mul_(torch.exp(-0.5 * Rnorm_clone))  # In-place operation
        weights.add_(1e-300)  # Add directly without creating new tensors
        weights.div_(weights.sum())  # Normalize in-place
        # Step 5: Resampling and updating particles
        indexes = systematic_resample_pytorch(weights)
        particles[:] = particles[indexes]
        # Step 6: Update mean
        mu = torch.sum(particles * weights.unsqueeze(1), dim=0)
        IntmediaPara = mu.clone() 
    
    ## EnSF with \bar{\gamma}^{L}_{i+1}
    IntmediaPara_lastEnSF = IntmediaPara.clone()
    
    diffusion_ref_pertb = RF_1Frac_DA.compute_diffusion_single(IntmediaPara_lastEnSF[0],\
                                                               IntmediaPara_lastEnSF[1], TriMesh_Subdom, device)
    for k in range(2):
        MatAEnSF[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dtEnSF, diffusion_ref_pertb[k], device)
        Local_Mat_pertb[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatAEnSF[k], MatB[k], MatC[k], dtEnSF)

    
    kflastDA1 = IntmediaPara_lastEnSF[2]*2000
    
    diff_frt_normallastDA = torch.zeros(ny, dtype=torch.float64)
    diff_frt_tangentlastDA = torch.zeros(ny, dtype=torch.float64)

    # Assign values to segments of diff_frt
    diff_frt_normallastDA[:ny // 4] = 1 / kflastDA1
    diff_frt_normallastDA[ny // 4:3 * ny // 4] = kflastDA1
    diff_frt_normallastDA[3 * ny // 4:] = 1 / kflastDA1
    
    diff_frt_tangentlastDA[:ny // 4] = kflastDA1
    diff_frt_tangentlastDA[ny // 4:3 * ny // 4] = 1 / kflastDA1
    diff_frt_tangentlastDA[3 * ny // 4:] = kflastDA1
    
    # Calculate keppa and alp_frt
    keppalastDA = (2 * diff_frt_normallastDA) / frt_length
    alp_frtlastDA = diff_frt_tangentlastDA * frt_length

    Glob_Mat_pertb = \
    RF_1Frac_DA.StiffMax_PureDiff_Vectorized_General(TriMesh_Subdom, h, node_frt, ny, \
                                                     Local_Mat_pertb, alp_frtlastDA, keppalastDA, dtEnSF, \
                                                     poros_frt, device)

    noise1 = np.sqrt(dtEnSF)*SDE_sigma*torch.randn_like(torch.zeros((NumEms[0]+NumEms[1]+ny, ensemble_size),\
                                                                    dtype=torch.float64, device=device))

    x0filterT_last = torch.transpose(x0EnSF.clone(), 0, 1)
    sln_state = \
        RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA_General_Aniso(x0filterT_last, ny, node_frt, TriMesh_Subdom, \
                                                                       ntEnSF, dtEnSF, poros_subdom, poros_frt, BCs_frt, \
                                                                       BCs_Subdom, Glob_Mat_pertb, GAUSSQUAD, Eqn_Subdom,\
                                                                       EqnBC_fracture, SDE_sigma, device, noise1, 0)
    
    x_state = torch.transpose(sln_state, 0, 1)+\
                    np.sqrt(dtEnSF)*SDE_Sigma*torch.randn_like(x_state, dtype=torch.float64, device=device)
    
    x_state = RF_1Frac_DA.RearrangeSol(x_state, TriMesh_Subdom, ny, node_frt)
    
    sln_bar = reverse_SDE(DOFs_Local, NumEgs, NumEms, NeumannEgs, num_indices, select_set, idA_sub,\
                          x0=x_state[:, spa_indices], time_steps=euler_steps, C=1, \
                          score_likelihood=score_likelihood)
    x_state[:, spa_indices] = sln_bar.clone()

    x_state[:, TotalEgs + torch.arange(0, NumEms[0])] =\
                torch.clamp(x_state[:, TotalEgs + torch.arange(0, NumEms[0])],\
                            min=0, max = 1.45)
    
    x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])] =\
                torch.clamp(x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])],\
                            min=0, max = 2.99999)

    x_state[:, TotalEgs + NumEms[0] + NumEms[1]+ torch.arange(0, ny)] =\
                torch.clamp(x_state[:, TotalEgs + NumEms[0] + NumEms[1]+ torch.arange(0, ny)],\
                            min=0, max = BCs_frt)
    
    x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min =-6, max = 6)
    x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]] = torch.clamp(x_state[:, NumEgs[0]:NumEgs[0]+NumEgs[1]], min=-6, max=6)
    x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)] = \
                    torch.clamp(x_state[:, NumEgs[0]+NumEgs[1]+torch.arange(0, ny+1)],\
                                min=-0.75, max = 9e-05)
        
    x_state[:, NeumannEgs[0]] = 0
    x_state[:, NumEgs[0]+NeumannEgs[1]] = 0
        
    x_state = RF_1Frac_DA.RearrangeSol_Reverse(x_state, TriMesh_Subdom, ny, node_frt)
    
    x_statebarEnSF = torch.zeros_like(est_save[[i+1], :])
    x_statebarEnSF += torch.mean(x_state.clone(),dim=0) #\bar{X^{L}}_{i+1} = \bar{X}_{i+1}
    
    
    est_save[[i+1], :] += torch.mean(x_state.clone(),dim=0)
    xs.append(mu)
    diff = torch.sqrt(torch.mean((est_save[[i+1], :]  - x_statebarEnSF)**2)).item()
    rmse_temp = torch.sqrt(torch.mean((est_save[[i+1], :]  - state_EnSF[[i+1], :])**2)).item()
    rmse = torch.sqrt(torch.mean((x_statebarEnSF  - state_EnSF[[i+1], :])**2)).item()
    
    # get time
    if x_state.device.type == 'cuda':
        torch.cuda.current_stream().synchronize()
    t2 = time.time()
    print(f'\t RMSE = {rmse_temp:.4f}')
    print(f'\t time = {t2-t1:.4f} ')

    # save information
    rmse_all.append(rmse_temp)
    # break
    if rmse_temp > 1000:
        print('diverge!')
        break

state_savepy  = state_ref.clone()
est_savepy = est_save.clone()

state_savepy = state_savepy.cpu().numpy()
est_savepy = est_savepy.cpu().numpy()
rmse_all = np.array(rmse_all)

scipy.io.savemat('UnitedFilter_2DRft_50PartialOb_Mixed_Aniso_Noise0001_v1.mat',\
                 {'Est_State':est_savepy, 'ExactState':state_savepy, 'rmse':rmse_all,\
                 'nx':nx, 'ny':ny, 'nttrue': nt, 'ntEnSF': ntEnSF, 'ApprPara':xs})
# scipy.io.savemat('UnitedFilter_2DRft_50PartialOb_Mixed_Aniso_Noise001_v1.mat',\
#                  {'Est_State':est_savepy, 'ExactState':state_savepy, 'rmse':rmse_all,\
#                  'nx':nx, 'ny':ny, 'nttrue': nt, 'ntEnSF': ntEnSF, 'ApprPara':xs})
# scipy.io.savemat('UnitedFilter_2DRft_50PartialOb_Mixed_Aniso_Noise01_v1.mat',\
#                  {'Est_State':est_savepy, 'ExactState':state_savepy, 'rmse':rmse_all,\
#                  'nx':nx, 'ny':ny, 'nttrue': nt, 'ntEnSF': ntEnSF, 'ApprPara':xs})